# LoRA token classification 예제 - BioNLP2004 데이터셋
## Reference : https://huggingface.co/docs/peft/task_guides/token-classification-lora

# 필요한 라이브러리 설치

In [ ]:
!pip install -q peft transformers datasets evaluate seqeval

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 190.9/190.9 kB 6.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 510.5/510.5 kB 20.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 10.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.6/43.6 kB 6.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 280.0/280.0 kB 20.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 15.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 18.3 MB/s eta 0:00:00


# 설정값 지정

In [ ]:
from datasets import load_dataset
from transformers import (   # 트랜스포머의 필요한 모듈
    AutoModelForTokenClassification,
    AutoTokenizer,
    DataCollatorForTokenClassification,
    TrainingArguments,
    Trainer,
)
from peft import get_peft_config, PeftModel, PeftConfig, get_peft_model, LoraConfig, TaskType  # peft 모델 불러옴
import evaluate
import torch
import numpy as np

#model_checkpoint = "roberta-large"
model_checkpoint = "roberta-base"   # 빠른 학습을 위해 base 모델로 설정
lr = 1e-3
batch_size = 16
#num_epochs = 10
num_epochs = 1      # 빠른 학습을 위해 epoch을 1로 설정

# BioNLP2004 데이터셋 불러오기
## (DNA, RNA, proteins 같은 생물학적 구조에 대한 tag를 나타내는 데이터셋입니다.)

목표: 단어 입력 -> BIO 구조에 대응되는 단어인지 토큰 단위로 Classfication

In [ ]:
# tags는 label id를 나타냅니다.
bionlp = load_dataset("tner/bionlp2004")
bionlp["train"][0]  # 단어에 대한 레이블이 대응되는 상태

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Generating train split:   0%|          | 0/16619 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/1927 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/3856 [00:00<?, ? examples/s]

{'tokens': ['Since',
  'HUVECs',
  'released',
  'superoxide',
  'anions',
  'in',
  'response',
  'to',
  'TNF',
  ',',
  'and',
  'H2O2',
  'induces',
  'VCAM-1',
  ',',
  'PDTC',
  'may',
  'act',
  'as',
  'a',
  'radical',
  'scavenger',
  '.'],
 'tags': [0, 7, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 3, 0, 0, 0, 0, 0, 0, 0, 0, 0]}

In [ ]:
# precision, accuracy, F1, and recall 등 sequence labeling tasks evaluation을 위한 모듈
seqeval = evaluate.load("seqeval")

In [ ]:
# 정답 Label 리스트
label_list = [
    "O",
    "B-DNA",
    "I-DNA",
    "B-protein",
    "I-protein",
    "B-cell_type",
    "I-cell_type",
    "B-cell_line",
    "I-cell_line",
    "B-RNA",
    "I-RNA",
]

In [ ]:
# 성능 측정을 위한 함수
def compute_metrics(p):
    predictions, labels = p
    predictions = np.argmax(predictions, axis=2)

    true_predictions = [
        [label_list[p] for (p, l) in zip(prediction, label) if l != -100]
        for prediction, label in zip(predictions, labels)
    ]
    true_labels = [
        [label_list[l] for (p, l) in zip(prediction, label) if l != -100]
        for prediction, label in zip(predictions, labels)
    ]

    results = seqeval.compute(predictions=true_predictions, references=true_labels)
    return {
        "precision": results["overall_precision"],
        "recall": results["overall_recall"],
        "f1": results["overall_f1"],
        "accuracy": results["overall_accuracy"],
    }

# Tokenizer 불러오기

In [ ]:
# roberta-base 모델에서 사용한 토크나이저
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint, add_prefix_space=True)

tokenizer_config.json:   0%|          | 0.00/25.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/481 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

In [ ]:
# 텍스트 데이터를 토큰 단위로 분리하는 함수
def tokenize_and_align_labels(examples):
    tokenized_inputs = tokenizer(examples["tokens"], truncation=True, is_split_into_words=True)

    labels = []
    for i, label in enumerate(examples[f"tags"]):
        word_ids = tokenized_inputs.word_ids(batch_index=i)
        previous_word_idx = None
        label_ids = []
        for word_idx in word_ids:
            if word_idx is None:
                label_ids.append(-100)
            elif word_idx != previous_word_idx:
                label_ids.append(label[word_idx])
            else:
                label_ids.append(-100)
            previous_word_idx = word_idx
        labels.append(label_ids)

    tokenized_inputs["labels"] = labels
    return tokenized_inputs

# 토크나이징

In [ ]:
# 불러온 데이터셋 전체에 토크나이징 진행
tokenized_bionlp = bionlp.map(tokenize_and_align_labels, batched=True)

Map:   0%|          | 0/16619 [00:00<?, ? examples/s]

Map:   0%|          | 0/1927 [00:00<?, ? examples/s]

Map:   0%|          | 0/3856 [00:00<?, ? examples/s]

In [ ]:
# 가장 긴 길이의 데이터에 맞게 padding -> 문장 길이에 따라 분리된 토큰의 개수가 다르기 때문
data_collator = DataCollatorForTokenClassification(tokenizer=tokenizer)

In [ ]:
# 정답 레이블별 int 할당
id2label = {   #  {key:value} = {int, label}
    0: "O",
    1: "B-DNA",
    2: "I-DNA",
    3: "B-protein",
    4: "I-protein",
    5: "B-cell_type",
    6: "I-cell_type",
    7: "B-cell_line",
    8: "I-cell_line",
    9: "B-RNA",
    10: "I-RNA",
}
label2id = {   #  {key:value} = {label, int}
    "O": 0,
    "B-DNA": 1,
    "I-DNA": 2,
    "B-protein": 3,
    "I-protein": 4,
    "B-cell_type": 5,
    "I-cell_type": 6,
    "B-cell_line": 7,
    "I-cell_line": 8,
    "B-RNA": 9,
    "I-RNA": 10,
}

# 허깅페이스 제공 RoBERTa -base 모델 다운로드
model = AutoModelForTokenClassification.from_pretrained(
    model_checkpoint, num_labels=11, id2label=id2label, label2id=label2id
)

Some weights of RobertaForTokenClassification were not initialized from the model checkpoint at roberta-base and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


# PEFT 모델 설정

In [ ]:
peft_config = LoraConfig(
    task_type=TaskType.TOKEN_CLS, inference_mode=False, r=16, lora_alpha=16, lora_dropout=0.1, bias="all"
    # task_type: 토큰 classification, 추론 모드: 아님, 로라 로우 랭크: 16, ..
)

# LoRA 기법으로 인해 전체 모델의 0.55%의 파라미터만 Fine-Tuning에 사용

In [ ]:
model = get_peft_model(model, peft_config)  # Roberta-base 모델에 LoRA config 적용
model.print_trainable_parameters()   # LoRA 적용 후 파라미터가 0.55 퍼센트로 줄어듬

trainable params: 691,990 || all params: 124,661,782 || trainable%: 0.5550939421032823


# Training config 설정

In [ ]:
training_args = TrainingArguments(
    #output_dir="roberta-large-lora-token-classification",
    output_dir="roberta-base-lora-token-classification",  # 출력 결과를 저장할 폴더명
    learning_rate=lr,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    num_train_epochs=num_epochs,
    weight_decay=0.01,
    evaluation_strategy="epoch",
    save_strategy="epoch",
    load_best_model_at_end=True,
)

# Training 시작

In [ ]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_bionlp["train"],
    eval_dataset=tokenized_bionlp["validation"],
    tokenizer=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics,
)

trainer.train()

Epoch,Training Loss,Validation Loss,Precision,Recall,F1,Accuracy
1,0.208700,0.174665,0.732550,0.786242,0.758447,0.941189


TrainOutput(global_step=1039, training_loss=0.2469388523046734, metrics={'train_runtime': 6414.5857, 'train_samples_per_second': 2.591, 'train_steps_per_second': 0.162, 'total_flos': 661020489129132.0, 'train_loss': 0.2469388523046734, 'epoch': 1.0})

# 학습결과 zip 파일로 압축후 다운로드

In [ ]:
import zipfile
import shutil
from google.colab import files

# 압축할 폴더 이름
folder_name = "roberta-base-lora-token-classification"

# 생성될 ZIP 파일 이름
zip_file_name = "roberta-base-lora-token-classification.zip"

# 폴더를 ZIP 파일로 압축
shutil.make_archive(zip_file_name[:-4], 'zip', folder_name)

# ZIP 파일을 로컬로 다운로드
files.download(zip_file_name)

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

# 학습결과 불러오기

### BioNLP2004 데이터셋에 1epoch Fine-Tuning 완료된 zip 파일 :
roberta-base-lora-token-classification.zip https://drive.google.com/file/d/1_mEcACHQkKcTEFpOcbSjEaPgDRBasGb0/view?usp=sharing


In [ ]:
# 직접 학습하지않고 학습이 완료된 zip 파일을 업로드해서 사용하고 싶은 경우에 주석을 해제해서 사용하세요!

# import zipfile

# # 압축 해제할 ZIP 파일 이름
# zip_file_name = "roberta-base-lora-token-classification.zip"

# # 압축을 해제할 대상 폴더. 이 예시에서는 같은 이름의 폴더에 압축을 해제합니다.
# extract_folder_name = "./roberta-base-lora-token-classification"

# # ZIP 파일 압축 해제
# with zipfile.ZipFile(zip_file_name, 'r') as zip_ref:
#     zip_ref.extractall(extract_folder_name)

# Inference를 위해 학습이 끝난 모델 Load하기

In [ ]:
#peft_model_id = "stevhliu/roberta-large-lora-token-classification"
peft_model_id = "./roberta-base-lora-token-classification/checkpoint-1039"   # 학습이 끝난 LoRA check point path 지정
config = PeftConfig.from_pretrained(peft_model_id)   # 학습된 모델의 config 지정
inference_model = AutoModelForTokenClassification.from_pretrained(   # freezing된 pre-trained RoBERTa 모델 불러오기
    config.base_model_name_or_path, num_labels=11, id2label=id2label, label2id=label2id
)
tokenizer = AutoTokenizer.from_pretrained(config.base_model_name_or_path)  # 학습된 모델의 토크나이저 불러옴
model = PeftModel.from_pretrained(inference_model, peft_model_id)  # inference용 모델로 freezing된 모델 + 학습이 끝난 LoRA check point path 지정
# freezing된 체크포인트와 LoRA로 학습한 체크포인트 a,b 를 불러오고 합쳐서 최종 모델

Some weights of RobertaForTokenClassification were not initialized from the model checkpoint at roberta-base and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


# sample text에 대한 Inference

In [ ]:
# 샘플 텍스트
text = "The activation of IL-2 gene expression and NF-kappa B through CD28 requires reactive oxygen production by 5-lipoxygenase."
# 토크나이징
inputs = tokenizer(text, return_tensors="pt")

In [ ]:
with torch.no_grad():
    logits = model(**inputs).logits

tokens = inputs.tokens()
predictions = torch.argmax(logits, dim=2)  # label 중 가장 확률 값이 높은 것을 추출하여 확정 label

for token, prediction in zip(tokens, predictions[0].numpy()):   # 예시 문장에 대해 토큰 별로 가장 확률이 높은 Label 출력
    print((token, model.config.id2label[prediction]))

('<s>', 'O')
('The', 'O')
('Ġactivation', 'O')
('Ġof', 'O')
('ĠIL', 'B-DNA')
('-', 'I-DNA')
('2', 'I-DNA')
('Ġgene', 'I-DNA')
('Ġexpression', 'O')
('Ġand', 'O')
('ĠNF', 'B-protein')
('-', 'I-protein')
('k', 'I-protein')
('appa', 'I-protein')
('ĠB', 'I-protein')
('Ġthrough', 'O')
('ĠCD', 'B-protein')
('28', 'I-protein')
('Ġrequires', 'O')
('Ġreactive', 'O')
('Ġoxygen', 'O')
('Ġproduction', 'O')
('Ġby', 'O')
('Ġ5', 'B-protein')
('-', 'I-protein')
('lip', 'I-protein')
('oxy', 'I-protein')
('gen', 'I-protein')
('ase', 'I-protein')
('.', 'O')
('</s>', 'O')
